
# Stirred Reactor (CSTR) with RMG Ammonia Mechanism in Cantera

This notebook demonstrates how to run a well-stirred, isothermal reactor (CSTR/JSR-style) using the RMG-exported Cantera YAML for ammonia oxidation. It builds on the duplicate-reaction fixer and species-name helper from the other tutorials, then walks through the core concepts: residence time, inlet conditions, and the auxiliary controllers Cantera uses to represent flow and pressure boundaries.



## Key concepts
- **Residence time (\(\tau\))**: the average time material spends in the reactor. For a gas-filled reactor, \(\tau = m / \dot{m} = \rho V / \dot{m}\), where \(m\) is mass in the reactor, \(\rho\) density, \(V\) volume, and \(|\dot{m}|\) the mass flow rate in/out.
- **Steady state**: in a CSTR, the inlet mass flow equals the outlet mass flow; we integrate in time until the composition stops changing appreciably.
- **Boundary hardware (modeled in Cantera)**: a supply reservoir feeds a mass-flow controller (sets \(\dot{m}\)), the well-stirred reactor holds the gas, and a downstream pressure controller + exhaust reservoir keep pressure fixed while letting mass leave.



## What you need
- Run from the `RMG-Py` repository root.
- Dependencies: `cantera`, `pyyaml`, `numpy`, `matplotlib`.
- We reuse the duplicate-reaction fixer (`fix_cantera`) so Cantera can load the RMG mechanism cleanly.


In [ ]:
"""
t3 utils fix_cantera module
A module to automatically fix issues with RMG-generated Cantera files, mainly resolving mislabeled duplicate reactions.
"""
import os

from typing import List, Optional, Union

import shutil
import time
import traceback
import yaml
import cantera as ct


def globalize_paths(path: str, project_directory: str) -> str:
    """
    Convert a relative path into an absolute path based on project_directory.
    If path is already absolute, return it unchanged.
    """
    if os.path.isabs(path):
        return path
    return os.path.normpath(os.path.join(project_directory, path))


def read_yaml_file(path: str,
                   project_directory: Optional[str] = None,
                   ) -> Union[dict, list]:
    """
    Read a YAML file (usually an input / restart file, but also conformers file)
    and return the parameters as python variables.

    Args:
        path (str): The YAML file path to read.
        project_directory (str, optional): The current project directory to rebase upon.

    Returns: Union[dict, list]
        The content read from the file.
    """
    if project_directory is not None:
        path = globalize_paths(path, project_directory)
    if not isinstance(path, str):
        raise InputError(f'path must be a string, got {path} which is a {type(path)}')
    if not os.path.isfile(path):
        raise InputError(f'Could not find the YAML file {path}')
    with open(path, 'r') as f:
        content = yaml.load(stream=f, Loader=yaml.FullLoader)
    return content

def to_yaml(py_content: Union[list, dict]) -> str:
    """
    Convert a Python list or dictionary to a YAML string format.

    Args:
        py_content (list, dict): The Python content to save.

    Returns: str
        The corresponding YAML representation.
    """
    yaml.add_representer(str, string_representer)
    yaml_str = yaml.dump(data=py_content)
    return yaml_str


def string_representer(dumper, data):
    """
    Add a custom string representer to use block literals for multiline strings.
    """
    if len(data.splitlines()) > 1:
        return dumper.represent_scalar(tag='tag:yaml.org,2002:str', value=data, style='|')
    return dumper.represent_scalar(tag='tag:yaml.org,2002:str', value=data)

def save_yaml_file(path: str,
                   content: Union[list, dict],
                   ) -> None:
    """
    Save a YAML file (usually an input / restart file, but also conformers file).

    Args:
        path (str): The YAML file path to save.
        content (list, dict): The content to save.
    """
    if not isinstance(path, str):
        raise InputError(f'path must be a string, got {path} which is a {type(path)}')
    yaml_str = to_yaml(py_content=content)
    if '/' in path and os.path.dirname(path) and not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(yaml_str)


def get_traceback(model_path: str) -> Optional[str]:
    """
    Try loading the Cantera model and return the traceback if it fails.

    Args:
        model_path (str): The path to the cantera YAML model file.

    Returns:
        Optional[str]: The traceback if the model fails to load.
    """
    tb = None
    try:
        ct.Solution(model_path)
    except ct.CanteraError:
        tb = traceback.format_exc()
    return tb


def fix_cantera(model_path: str):
    """
    Fix a Cantera model that has incorrectly marked duplicate reactions.
    Creates a backup copy of the Cantera model and fixes the content of the original file in place.

    Args:
        model_path (str): The path to the cantera YAML model file.

    Returns:
        bool: Whether the model was fixed.
    """
    shutil.copyfile(model_path, model_path + '.bak')
    done, fixed = False, False
    counter = 0
    while not done and counter < 1000:
        counter += 1
        tb = get_traceback(model_path)
        if tb is None:
            done = True
            break
        else:
            if 'Undeclared duplicate reactions detected' in tb:
                fix_undeclared_duplicate_reactions(model_path, tb)
                fixed = True
            elif 'No duplicate found for declared duplicate reaction' in tb:
                fix_no_duplicate_found(model_path, tb)
                fixed = True
            else:
                print(f'Could not fix {model_path}:\n\n{tb}')
                break
        time.sleep(1)
    if fixed:
        print(f'Fixing Cantera model {model_path} (and creating a backup copy with a .bak extension).')
    else:
        os.remove(model_path + '.bak')
    return done


def fix_undeclared_duplicate_reactions(model_path: str, tb: str):
    """
    Fix a Cantera model that has undeclared duplicate reactions.

    Args:
        model_path (str): The path to the cantera YAML model file.
        tb (str): The traceback.
    """
    content = read_yaml_file(model_path)
    rxns = get_dup_rxn_indices(tb)
    print(f'Marking reactions {", ".join([str(r) for r in rxns])} as duplicate.')
    for i in rxns:
        content['reactions'][i - 1]['duplicate'] = True
    save_yaml_file(model_path, content)


def fix_no_duplicate_found(model_path: str, tb: str):
    """
    Fix a Cantera model that has a reaction marked as duplicate by mistake with no other duplicate reaction.

    Args:
        model_path (str): The path to the cantera YAML model file.
        tb (str): The traceback.
    """
    content = read_yaml_file(model_path)
    rxns = get_mistakenly_marked_dup_rxns(tb)
    for i in rxns:
        if 'duplicate' in content['reactions'][i].keys():
            print(f'Marking reaction {i} as non-duplicate.')
            del content['reactions'][i]['duplicate']
    save_yaml_file(model_path, content)


def get_dup_rxn_indices(tb: str) -> List[int]:
    """
    Get the duplicate reactions from the traceback.

    Args:
        tb (str): The traceback.

    Returns:
        List[int]: The reactions indices.
    """
    rxns = list()
    if tb is None:
        return rxns
    lines = tb.split('\n')
    read = False
    for line in lines:
        if 'Undeclared duplicate reactions detected:' in line:
            read = True
        if '|  Line |' in line:
            break
        if read and 'Reaction' in line:
            rxns.append(int(line.split()[1].split(':')[0]))
    return rxns


def get_mistakenly_marked_dup_rxns(tb: str) -> List[int]:
    """
    Get the duplicate reactions from the traceback.

    Args:
        tb (str): The traceback.

    Returns:
        List[int]: The reactions indices.
    """
    rxns = list()
    if tb is None:
        return rxns
    lines = tb.split('\n')
    for line in lines:
        if 'No duplicate found for declared duplicate reaction number' in line:
            rxns.append(int(line.split()[8]))
    return rxns


## Fix the RMG-generated Cantera file (optional)
If Cantera raises duplicate-reaction bookkeeping errors, the helper below will patch the YAML in place and keep a `.bak` copy. If the file loads cleanly, nothing is changed.


In [ ]:

model_path = "examples/rmg/ammonia/cantera/chem_annotated.yaml"

# Only modifies the YAML if Cantera reports duplicate bookkeeping issues.
_ = fix_cantera(model_path)



## Handle RMG species names
Species names include numeric identifiers (e.g., `NH3(1)`). The helper returns the full Cantera species name from a base label so you don't have to track the numbering manually.


In [ ]:

def get_species(gas, name):
    for s in gas.species_names:
        if s.startswith(name + "("):
            return s
    raise KeyError(f"Species {name} not found.")



## Define inlet conditions and residence time
- Set inlet temperature, pressure, and composition.
- Choose reactor volume and target residence time $\tau$.
- Compute the mass flow rate $ \dot{m} = \rho V / \tau $ so the residence time matches your design.


In [ ]:

import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import time

# Load mechanism
gas = ct.Solution(model_path)

# Inlet state
T0 = 1200.0  # K
P0 = ct.one_atm

NH3 = get_species(gas, "NH3")
O2 = get_species(gas, "O2")
N2 = get_species(gas, "N2")
NO = get_species(gas, "NO")

inlet = {NH3: 0.02, O2: 0.15, "Ar": 0.83}

gas.TPX = T0, P0, inlet

# Reactor sizing
reactor_volume = 78e-6  # m^3 (78 cm^3, as in the JSR example)
residence_time = 3.0    # seconds

rho0 = gas.density
mdot = rho0 * reactor_volume / residence_time  # kg/s to enforce tau
print(f"Mass flow rate set to {mdot:.4e} kg/s for tau = {residence_time}s")



## Build the stirred reactor system
Cantera models the flow boundaries explicitly:
- Upstream `Reservoir` represents the feed tank at the inlet state.
- `MassFlowController` sets \(\dot{m}\) into the reactor.
- `IdealGasReactor` is the well-stirred volume; `energy="off"` keeps it isothermal (use `"on"` for adiabatic).
- `PressureController` vents to a downstream `Reservoir` while holding the pressure.


In [ ]:

# Upstream/downstream reservoirs
fuel_air_mixture_tank = ct.Reservoir(gas)
exhaust = ct.Reservoir(gas)

# Stirred reactor
stirred_reactor = ct.IdealGasReactor(gas, energy="off", volume=reactor_volume)

# Controllers to impose flow and pressure
mass_flow_controller = ct.MassFlowController(
    upstream=fuel_air_mixture_tank,
    downstream=stirred_reactor,
    mdot=mdot,
)

pressure_regulator = ct.PressureController(
    upstream=stirred_reactor,
    downstream=exhaust,
    master=mass_flow_controller,
)

reactor_network = ct.ReactorNet([stirred_reactor])

# Storage for time history
state_history = ct.SolutionArray(gas, extra=["t"])
max_simulation_time = 50.0  # seconds of integration time

start = time.time()
t = 0.0
counter = 0
sample_every = 10  # store every nth step to keep arrays light

while t < max_simulation_time:
    t = reactor_network.step()
    counter += 1
    if counter % sample_every == 0:
        state_history.append(stirred_reactor.thermo.state, t=t)

end = time.time()
print(f"Simulation ran {counter} steps in {end - start:0.2f} s")



## Plot selected species vs. time
Here we look at NH3, NO, and N2 approaching steady state. Adjust `max_simulation_time` if the system is still evolving at the end of the integration.


In [ ]:

plt.figure(figsize=(8, 5))
plt.semilogx(state_history.t, state_history[NH3].X, label="NH3")
plt.semilogx(state_history.t, state_history[NO].X, label="NO")
plt.semilogx(state_history.t, state_history[N2].X, label="N2")

plt.xlabel("Time (s)")
plt.ylabel("Mole fraction")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



## Notes for exploration
- Change `residence_time` to see its impact; `mdot` will recompute from the chosen volume and inlet density.
- Switch to `energy="on"` for adiabatic behavior or add heat transfer terms for isothermal control.
- Modify `inlet` composition, `T0`, or `P0` to replicate other cases from the experimental article.
- Steady state: stop integrating when key species flatten out or when `reactor_network.step()` stops changing composition appreciably.
